# vorongen notes: API-first usage

This notebook shows custom YAML input and classic-vs-torch comparison.

In [ ]:
from vorongen import (
    RunConfig,
    compare_torch_vs_classic,
    generate,
    load_config,
)

In [ ]:
yaml_config = """
metadata:
  name: notebook_binary_demo
  version: "1.0"

columns:
  - column_id: "loyalty"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "bernoulli"
      probabilities: { true_prob: 0.4, false_prob: 0.6 }

  - column_id: "discount"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["loyalty"]
      conditional_probs:
        "loyalty=1": { true_prob: 0.65, false_prob: 0.35 }
        "loyalty=0": { true_prob: 0.20, false_prob: 0.80 }
"""

config = load_config(yaml_config)
run_cfg = RunConfig(
    n_rows=2000,
    seed=7,
    tolerance=0.03,
    max_attempts=2,
    log_level="quiet",
    collect_history=True,
)

In [ ]:
single = generate(config, run_cfg)
{
    "success": single.success,
    "objective": single.metrics["objective"],
    "max_error": single.metrics["max_error"],
}

In [ ]:
comparison = compare_torch_vs_classic(config, run_cfg)
{
    "classic_objective": comparison["classic"].metrics["objective"],
    "torch_objective": comparison["torch"].metrics["objective"],
    "classic_confidence": comparison["classic"].quality_report["confidence"],
    "torch_confidence": comparison["torch"].quality_report["confidence"],
}

In [ ]:
comparison["torch"].dataframe.head()